In [3]:
! pip install refinitiv.data
! pip install yfinance


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/16/bf/7c0c89ff8ba53592b9cb5157f70e90d8bbb04d60094fc4f10035e158b981/yfinance-0.2.66-py2.py3-none-any.whl.metadata
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/38/74/f94141b38a51a553efef7f510fc213894161ae49b88bffd037f8d2a7cb2f/frozendict-2.4.7-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
     - -------------------------------------- 0.1/3.0 MB 3.3 MB/s eta 0:00:01
     ------ --------------------------------- 0.5/3.0 MB 4.9 MB/s eta 0:00:01
     --------------------


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
import logging
import json
import refinitiv.data as rd
from typing import List, Dict, Tuple
from datetime import datetime
import warnings


In [6]:
"""
ESG-based stock selection and data fetching for pairs trading.
Uses Refinitiv Workspace for ESG data and Yahoo Finance for price data.
Automatically selects Brown/Green stocks from S&P 500 based on Environment scores.
Uses random sampling for stock selection.
"""

# Suppress warnings
warnings.filterwarnings('ignore')
logging.getLogger('yfinance').setLevel(logging.CRITICAL)

# Set random seed for reproducibility (optional - remove for true randomness)
random.seed(42)
np.random.seed(42)


class RefinitivESGSelector:
    """Select stocks based on ESG scores from S&P 500 using Refinitiv data."""
    
    def __init__(self, config_file: str = "refinitiv-data.config"):
        """
        Initialize selector with S&P 500 tickers and Refinitiv credentials.
        
        Args:
            config_file: Path to Refinitiv configuration file (default: refinitiv-data.config)
        """
        self.config_file = config_file
        self.sp500_tickers = self._get_sp500_tickers()
        self.energy_sectors = ['Energy', 'Utilities']
        self._initialize_refinitiv()
        print(f"Loaded {len(self.sp500_tickers)} S&P 500 tickers from GitHub")
        
    def _initialize_refinitiv(self):
        """Initialize Refinitiv Data Platform connection using existing config file."""
        try:
            # Open Refinitiv session using the config file
            # The file should already contain your credentials
            rd.open_session(config_name=self.config_file)
            
            print("✓ Successfully connected to Refinitiv Workspace")
            
        except FileNotFoundError:
            print(f"❌ ERROR: Configuration file '{self.config_file}' not found!")
            print(f"\nPlease ensure the file 'refinitiv-data.config' exists in the current directory.")
            raise
        except Exception as e:
            print(f"❌ ERROR: Failed to connect to Refinitiv: {e}")
            print("\nMake sure:")
            print("1. Refinitiv Workspace is running")
            print("2. You are logged into Workspace")
            print("3. The configuration file contains valid credentials")
            raise
        
    def _get_sp500_tickers(self) -> List[str]:
        """
        Fetch current S&P 500 constituents from GitHub dataset.
        
        Returns:
            List of ticker symbols
        """
        try:
            url = 'https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv'
            df = pd.read_csv(url)
            return df['Symbol'].str.replace('.', '-').tolist()
        except Exception as e:
            print(f"Error fetching S&P 500 list from GitHub: {e}")
            return []
    
    def _convert_ticker_to_ric(self, ticker: str) -> str:
        """
        Convert Yahoo ticker to Refinitiv RIC (Reuters Instrument Code).
        
        Args:
            ticker: Yahoo Finance ticker (e.g., 'AAPL')
        
        Returns:
            Refinitiv RIC (e.g., 'AAPL.O')
        """
        # Basic conversion - most US stocks on NYSE/NASDAQ
        # .O = NASDAQ, .N = NYSE, but we'll try .O first as fallback
        if '.' in ticker:
            return ticker  # Already has exchange suffix
        return f"{ticker}.O"  # Default to NASDAQ
    
    def fetch_esg_scores(self, tickers: List[str] = None) -> pd.DataFrame:
        """
        Fetch Environment scores from Refinitiv for given tickers.
        
        Args:
            tickers: List of ticker symbols (defaults to S&P 500)
        
        Returns:
            DataFrame with columns: [ticker, ric, sector, industry, environment_score]
        """
        if tickers is None:
            tickers = self.sp500_tickers
        
        print(f"Fetching Environment scores from Refinitiv for {len(tickers)} stocks...")
        print("This may take a few minutes...\n")
        
        # Convert tickers to RICs
        rics = [self._convert_ticker_to_ric(ticker) for ticker in tickers]
        
        esg_data = []
        success_count = 0
        fail_count = 0
        
        # Refinitiv ESG field codes
        # TR.EnvironmentPillarScore = Environment Pillar Score (0-100)
        # TR.TRESGScore = Overall ESG Score
        # TR.GICSSector = Sector
        # TR.GICSIndustry = Industry
        
        fields = [
            'TR.EnvironmentPillarScore',
            'TR.TRESGScore', 
            'TR.GICSSector',
            'TR.GICSIndustry'
        ]
        
        # Fetch data in batches (Refinitiv works better with batch requests)
        batch_size = 100
        
        for i in range(0, len(rics), batch_size):
            batch_rics = rics[i:i+batch_size]
            batch_tickers = tickers[i:i+batch_size]
            
            try:
                # Request data from Refinitiv
                df = rd.get_data(
                    universe=batch_rics,
                    fields=fields
                )
                
                if df is not None and not df.empty:
                    # Process each row
                    for idx, row in df.iterrows():
                        env_score = row.get('Environment Pillar Score', None)
                        
                        if env_score is not None and not pd.isna(env_score):
                            # Find corresponding ticker
                            ric = row.get('Instrument', '')
                            ticker_idx = batch_rics.index(ric) if ric in batch_rics else idx
                            ticker = batch_tickers[ticker_idx] if ticker_idx < len(batch_tickers) else 'Unknown'
                            
                            esg_data.append({
                                'ticker': ticker,
                                'ric': ric,
                                'sector': row.get('GICS Sector', 'Unknown'),
                                'industry': row.get('GICS Industry', 'Unknown'),
                                'environment_score': float(env_score),
                                'esg_score': row.get('ESG Score', None)
                            })
                            success_count += 1
                        else:
                            fail_count += 1
                
            except Exception as e:
                print(f"Warning: Error fetching batch {i//batch_size + 1}: {e}")
                fail_count += len(batch_rics)
            
            if (i + batch_size) % 200 == 0:
                print(f"Progress: {min(i+batch_size, len(rics))}/{len(rics)} | ✓ Success: {success_count} | ✗ No data: {fail_count}")
        
        result_df = pd.DataFrame(esg_data)
        
        print(f"\n{'='*60}")
        print(f"FETCHING COMPLETE")
        print(f"{'='*60}")
        print(f"Total stocks processed: {len(tickers)}")
        print(f"✓ Stocks with Environment Score: {success_count}")
        print(f"✗ Stocks without ESG data: {fail_count}")
        print(f"Success rate: {(success_count/len(tickers)*100):.1f}%")
        
        if len(result_df) == 0:
            print("\n⚠️  WARNING: No Environment scores found!")
            print("\nPossible reasons:")
            print("1. Refinitiv Workspace is not running")
            print("2. No data permissions for ESG content")
            print("3. Invalid RIC conversions")
        elif success_count < 30:
            print(f"\n⚠️  WARNING: Only {success_count} stocks found with ESG data.")
        
        return result_df
    
    def select_brown_green_stocks(self, 
                                 esg_df: pd.DataFrame,
                                 n_brown: int = 15,
                                 n_green: int = 15,
                                 energy_focus: bool = True,
                                 brown_percentile: float = 0.25,
                                 green_percentile: float = 0.75,
                                 random_seed: int = None) -> Dict[str, List[str]]:
        """
        Randomly select Brown and Green stocks from eligible pools based on Environment Score.
        
        Args:
            esg_df: DataFrame with Environment scores
            n_brown: Number of brown stocks to randomly select
            n_green: Number of green stocks to randomly select
            energy_focus: If True, focus on energy-related sectors
            brown_percentile: Select brown stocks from bottom X percentile (default: 0.25 = bottom 25%)
            green_percentile: Select green stocks from top X percentile (default: 0.75 = top 25%)
            random_seed: Set seed for reproducibility (None = truly random)
        
        Returns:
            Dictionary with 'brown' and 'green' ticker lists
        """
        # Set random seed if specified
        if random_seed is not None:
            random.seed(random_seed)
            np.random.seed(random_seed)
        
        if energy_focus:
            # Filter for energy-related sectors
            energy_df = esg_df[esg_df['sector'].isin(self.energy_sectors)].copy()
            
            if len(energy_df) < (n_brown + n_green):
                print(f"Warning: Only {len(energy_df)} energy stocks found. Using all sectors.")
                energy_df = esg_df.copy()
        else:
            energy_df = esg_df.copy()
        
        # Sort by Environment Score
        sorted_df = energy_df.sort_values('environment_score')
        
        # Calculate percentile thresholds
        brown_threshold = sorted_df['environment_score'].quantile(brown_percentile)
        green_threshold = sorted_df['environment_score'].quantile(green_percentile)
        
        # Create pools of eligible stocks
        brown_pool = sorted_df[sorted_df['environment_score'] <= brown_threshold]
        green_pool = sorted_df[sorted_df['environment_score'] >= green_threshold]
        
        print(f"\nBrown pool: {len(brown_pool)} stocks with Environment Score <= {brown_threshold:.2f}")
        print(f"Green pool: {len(green_pool)} stocks with Environment Score >= {green_threshold:.2f}")
        
        # Check if we have enough stocks
        if len(brown_pool) < n_brown:
            print(f"Warning: Only {len(brown_pool)} brown stocks available, requested {n_brown}")
            n_brown = len(brown_pool)
        
        if len(green_pool) < n_green:
            print(f"Warning: Only {len(green_pool)} green stocks available, requested {n_green}")
            n_green = len(green_pool)
        
        # Randomly select stocks from pools
        brown_stocks = random.sample(brown_pool['ticker'].tolist(), n_brown)
        green_stocks = random.sample(green_pool['ticker'].tolist(), n_green)
        
        # Get score ranges for selected stocks
        brown_scores = sorted_df[sorted_df['ticker'].isin(brown_stocks)]['environment_score']
        green_scores = sorted_df[sorted_df['ticker'].isin(green_stocks)]['environment_score']
        
        print(f"\n{'='*60}")
        print(f"RANDOMLY SELECTED {len(brown_stocks)} BROWN STOCKS (low Environment Score):")
        print(f"{'='*60}")
        for ticker in brown_stocks:
            score = sorted_df[sorted_df['ticker'] == ticker]['environment_score'].values[0]
            sector = sorted_df[sorted_df['ticker'] == ticker]['sector'].values[0]
            print(f"  {ticker:6s} - Score: {score:5.2f} - {sector}")
        print(f"\nBrown Environment Score range: {brown_scores.min():.2f} - {brown_scores.max():.2f}")
        
        print(f"\n{'='*60}")
        print(f"RANDOMLY SELECTED {len(green_stocks)} GREEN STOCKS (high Environment Score):")
        print(f"{'='*60}")
        for ticker in green_stocks:
            score = sorted_df[sorted_df['ticker'] == ticker]['environment_score'].values[0]
            sector = sorted_df[sorted_df['ticker'] == ticker]['sector'].values[0]
            print(f"  {ticker:6s} - Score: {score:5.2f} - {sector}")
        print(f"\nGreen Environment Score range: {green_scores.min():.2f} - {green_scores.max():.2f}")
        
        return {
            'brown': brown_stocks,
            'green': green_stocks,
            'brown_pool_size': len(brown_pool),
            'green_pool_size': len(green_pool),
            'brown_threshold': brown_threshold,
            'green_threshold': green_threshold
        }
    
    def close(self):
        """Close Refinitiv session."""
        try:
            rd.close_session()
            print("\n✓ Refinitiv session closed")
        except:
            pass


class ESGDataFetcher:
    """Fetch price data for ESG-selected stocks from Yahoo Finance."""
    
    def __init__(self, start_date: str = "2015-01-01", end_date: str = "2024-12-31"):
        self.start_date = start_date
        self.end_date = end_date
    
    def fetch_stock(self, ticker: str) -> pd.Series:
        """
        Download adjusted close price for a single stock.
        
        Args:
            ticker: Stock symbol
        
        Returns:
            Series with adjusted close prices
        """
        try:
            data = yf.download(ticker, start=self.start_date, end=self.end_date, 
                             auto_adjust=True, progress=False)
            return data['Close']
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
            return pd.Series(dtype=float)
    
    def fetch_multiple_stocks(self, tickers: List[str]) -> pd.DataFrame:
        """
        Download multiple stocks at once.
        
        Args:
            tickers: List of stock symbols
        
        Returns:
            DataFrame with one column per ticker
        """
        try:
            print(f"\nDownloading price data from Yahoo Finance for {len(tickers)} stocks...")
            data = yf.download(tickers, start=self.start_date, end=self.end_date,
                             auto_adjust=True, progress=False)['Close']
            
            # Handle single vs multiple tickers
            if isinstance(data, pd.Series):
                data = data.to_frame(tickers[0])
            
            df = data.dropna()
            print(f"✓ Successfully downloaded data: {df.shape[0]} days, {df.shape[1]} stocks")
            
            return df
        except Exception as e:
            print(f"Error fetching multiple stocks: {e}")
            return pd.DataFrame()
    
    def fetch_brown_green_data(self, stock_dict: Dict[str, List[str]]) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        Fetch data for both Brown and Green stocks separately.
        
        Args:
            stock_dict: Dictionary with 'brown' and 'green' ticker lists
        
        Returns:
            Tuple of (brown_df, green_df)
        """
        brown_df = self.fetch_multiple_stocks(stock_dict['brown'])
        green_df = self.fetch_multiple_stocks(stock_dict['green'])
        
        return brown_df, green_df


# Example usage workflow
def main():
    """
    Complete workflow: ESG selection from Refinitiv + price data from Yahoo Finance.
    """
    print("="*60)
    print("ESG-BASED STOCK SELECTION WORKFLOW")
    print("Using Refinitiv for ESG data & GitHub for S&P 500 list")
    print("="*60)
    
    try:
        # Step 1: Initialize selector with Refinitiv
        selector = RefinitivESGSelector(config_file="refinitiv-data.config")
        
        # Step 2: Fetch ESG scores from Refinitiv
        # For testing with a small subset:
        # test_tickers = selector.sp500_tickers[:50]
        # esg_df = selector.fetch_esg_scores(test_tickers)
        
        esg_df = selector.fetch_esg_scores()  # Full S&P 500
        
        # Check if we have enough data
        if len(esg_df) == 0:
            print("\n❌ ERROR: No ESG data available. Cannot proceed.")
            return None, None, None, None
        
        if len(esg_df) < 30:
            print(f"\n⚠️  WARNING: Only {len(esg_df)} stocks with ESG data found.")
        
        # Save Environment scores
        esg_df.to_csv('environment_scores_refinitiv.csv', index=False)
        print("\n✓ Environment scores saved to 'environment_scores_refinitiv.csv'")
        
        # Step 3: Select Brown and Green stocks RANDOMLY
        selected_stocks = selector.select_brown_green_stocks(
            esg_df, 
            n_brown=15, 
            n_green=15,
            energy_focus=True,
            brown_percentile=0.25,  # Select from worst 25%
            green_percentile=0.75,   # Select from best 25%
            random_seed=None        # Set to integer for reproducibility
        )
        
        # Step 4: Fetch price data from Yahoo Finance
        fetcher = ESGDataFetcher(start_date="2015-01-01", end_date="2024-12-31")
        brown_df, green_df = fetcher.fetch_brown_green_data(selected_stocks)
        
        # Step 5: Save price data
        brown_df.to_csv('brown_stocks_prices.csv')
        green_df.to_csv('green_stocks_prices.csv')
        
        print("\n" + "="*60)
        print("WORKFLOW COMPLETE")
        print("="*60)
        print(f"Brown stocks data: {brown_df.shape}")
        print(f"Green stocks data: {green_df.shape}")
        print("\nFiles saved:")
        print("- environment_scores_refinitiv.csv")
        print("- brown_stocks_prices.csv")
        print("- green_stocks_prices.csv")
        
        # Close Refinitiv session
        selector.close()
        
        return esg_df, selected_stocks, brown_df, green_df
        
    except Exception as e:
        print(f"\n❌ ERROR in workflow: {e}")
        return None, None, None, None


if __name__ == "__main__":
    # Run the complete workflow
    esg_df, selected_stocks, brown_df, green_df = main()
    
    # Optional: Display summary statistics
    if esg_df is not None and not esg_df.empty:
        print("\n" + "="*60)
        print("ENVIRONMENT SCORE SUMMARY STATISTICS")
        print("="*60)
        print(esg_df.groupby('sector')['environment_score'].describe())

ESG-BASED STOCK SELECTION WORKFLOW
Using Refinitiv for ESG data & GitHub for S&P 500 list
❌ ERROR: Failed to connect to Refinitiv: Cannot open session {'password': '******', 'username': 'francesco.mammetti@studenti.luiss.it'}
This session is not defined in the refinitiv-data.config configuration file

Make sure:
1. Refinitiv Workspace is running
2. You are logged into Workspace
3. The configuration file contains valid credentials

❌ ERROR in workflow: Cannot open session {'password': '******', 'username': 'francesco.mammetti@studenti.luiss.it'}
This session is not defined in the refinitiv-data.config configuration file
